# **SET UP**

In [ ]:
!pip install SciencePlots

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
import scienceplots

from scipy import stats
from scipy.stats import t, linregress
from matplotlib.dates import date2num, num2date
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import datetime
from datetime import datetime, date, timedelta
import folium
import branca.colormap
from geopy.geocoders import Nominatim

# **Newly updated MCL function**

In [ ]:
class DataAnalyzer:
    def __init__(self, data_or_url):
        if isinstance(data_or_url, str):  # Assuming file path is a string
            self.data_url = data_or_url
            self.data = self.load_data()
        elif isinstance(data_or_url, pd.DataFrame):  # Assuming DataFrame
            self.data = data_or_url
        else:
            raise ValueError("Input must be a file path or a pandas DataFrame")

    def load_data(self):
        file_extension = self.data_url.split('.')[-1]
        if file_extension == 'csv':
            return pd.read_csv(self.data_url)
        elif file_extension in ['xlsx', 'xlsm', 'xltx', 'xltm']:  # Add more excel file extensions if needed
            return pd.read_excel(self.data_url)
        else:
            raise ValueError("Unsupported file format")

    def print_column_names(self):
        print("Column names in the dataset:", self.data.columns.tolist())

    def preprocess_data(self, well_name_column, analyte_name_column, well_name, analyte_name):
        filtered_data = self.data[(self.data[well_name_column] == well_name) & (self.data[analyte_name_column] == analyte_name)]
        filtered_data = filtered_data.dropna(subset=['COLLECTION_DATE', 'RESULT'])
        filtered_data['COLLECTION_DATE'] = pd.to_datetime(filtered_data['COLLECTION_DATE']).dt.date
        filtered_data['RESULT_LOG'] = np.log10(filtered_data['RESULT'])
        return filtered_data

    def remove_outliers(self, data, result_column='RESULT_LOG', z_threshold=4):
        z = np.abs(stats.zscore(data[result_column]))
        return data[(z < z_threshold)]

    def predict_time_at_MCL(self, slope, intercept, log_MCL, earliest_date):
        if slope >= 0:
            return None
        time_at_MCL = (log_MCL - intercept) / slope
        try:
            time_at_MCL_date = num2date(time_at_MCL).replace(tzinfo=None).date()
            if earliest_date.year < 1 or time_at_MCL_date.year >= 10000 or time_at_MCL_date <= earliest_date:
                return None
            return time_at_MCL_date
        except ValueError:
            return None

    def adjust_date_based_on_confidence(self, base_date, slope, confidence_interval, days_per_unit):
        if base_date is None or slope == 0:
            return None, None, None
        days_adjustment = confidence_interval / abs(slope) * days_per_unit
        upper_date = base_date + timedelta(days=days_adjustment) if base_date else None
        lower_date = base_date - timedelta(days=days_adjustment) if base_date else None
        return lower_date, upper_date

    def generate_analyte_summary(self, analyte_name_column, analyte_name, well_name_column, MCL):
        unique_wells = self.data[self.data[analyte_name_column] == analyte_name][well_name_column].unique()
        summary = []

        for well in unique_wells:
            query = self.preprocess_data(well_name_column, analyte_name_column, well, analyte_name)
            if query.empty or len(query['COLLECTION_DATE'].unique()) < 2:
                continue

            x = date2num(query['COLLECTION_DATE'])
            y = query['RESULT_LOG']
            slope, intercept, r_value, p_value, std_err = linregress(x, y)

            if slope > 0:
                trend = 'Increase'
            elif slope < 0:
                trend = 'Decrease'
            else:
                trend = 'Neutral'

            if trend == 'Neutral':
                continue

            log_MCL = np.log10(MCL)
            time_at_MCL = self.predict_time_at_MCL(slope, intercept, log_MCL, min(query['COLLECTION_DATE']))

            # Calculate total decrease duration (for wells with decreasing trends)
            total_decrease_duration = None
            if trend == 'Decrease' and time_at_MCL:
                total_decrease_duration = (time_at_MCL - min(query['COLLECTION_DATE'])).days

            df = len(x) - 2
            ts = t.ppf(1 - 0.025, df)  # Two-sided t-critical value for 95% CI
            ci_slope = ts * std_err

            lower_date, upper_date = None, None
            if time_at_MCL and isinstance(time_at_MCL, date):
                lower_date, upper_date = self.adjust_date_based_on_confidence(time_at_MCL, slope, ci_slope, 365.25)

            summary.append({
                'well_name': well,
                'slope': slope,
                'p_value': p_value,  # Add p-value to summary
                'trend': trend,
                'time@MCL': time_at_MCL,
                'lower_time@MCL': lower_date,
                'upper_time@MCL': upper_date,
                'total_decrease_duration': total_decrease_duration
            })

        summary_df = pd.DataFrame(summary)
        return summary_df

    def compile_results(self, well_analyte_pairs, well_name_column, analyte_name_column, MCL):
        results = []
        for well_name, analyte_name in well_analyte_pairs:
            time_at_MCL = None
            query = self.preprocess_data(well_name_column, analyte_name_column, well_name, analyte_name)
            if query.empty:
                continue

            query = self.remove_outliers(query)
            x = date2num(query['COLLECTION_DATE'])
            y = query['RESULT_LOG']

            if len(x) < 2:
                continue

            slope, intercept, r_value, p_value, std_err = linregress(x, y)
            df = len(x) - 2
            ts = t.ppf(1 - 0.025, df)

            trend = "Increase" if slope > 0 else "Decrease" if slope < 0 else "Neutral"
            log_MCL = np.log10(MCL) if MCL else None
            earliest_date = min(query['COLLECTION_DATE'])
            time_at_MCL = self.predict_time_at_MCL(slope, intercept, log_MCL, earliest_date) if MCL else None

            # Calculate total decrease duration (for wells with decreasing trends)
            total_decrease_duration = None
            if trend == 'Decrease' and time_at_MCL:
                total_decrease_duration = (time_at_MCL - min(query['COLLECTION_DATE'])).days

            lower_date, upper_date = None, None
            if time_at_MCL and isinstance(time_at_MCL, date):
                lower_date, upper_date = self.adjust_date_based_on_confidence(time_at_MCL, slope, ts * std_err, 365.25)

            results.append({
                'well_name': well_name,
                'slope': slope,
                'p_value': p_value,  # Include p-value in results
                'trend': trend,
                'time@MCL': time_at_MCL,
                'lower_time@MCL': lower_date,
                'upper_time@MCL': upper_date,
                'total_decrease_duration': total_decrease_duration  # Add total decrease duration
            })

        results_df = pd.DataFrame(results)
        results_df.to_csv('time2MCL_results.csv', index=False)
        return results_df

    def plot_MCL(self, well_name_column, analyte_name_column, well_name, analyte_name, year_interval=20, save_dir='plot_MCL', MCL=None, extrapolate_until_year=2025):
        plt.style.use(['science', 'no-latex'])

        # Set the global font size parameters
        plt.rcParams.update({
            'font.size': 18,
            'axes.titlesize': 18,
            'axes.labelsize': 18,
            'xtick.labelsize': 18,
            'ytick.labelsize': 18,
            'legend.fontsize': 18,
            'figure.titlesize': 18
        })

        query = self.preprocess_data(well_name_column, analyte_name_column, well_name, analyte_name)
        if query.empty:
            print(f'No results found for {well_name} and {analyte_name}')
            return

        query = self.remove_outliers(query, result_column='RESULT_LOG')
        x = date2num(query['COLLECTION_DATE'])
        y = query['RESULT_LOG']

        if len(x) < 2:
            print("Not enough data for regression.")
            return

        slope, intercept, r_value, p_value, std_err = linregress(x, y)
        df = len(x) - 2
        ts = t.ppf(1 - 0.025, df)

        log_MCL = np.log10(MCL) if MCL else None
        time_at_MCL = self.predict_time_at_MCL(slope, intercept, log_MCL, min(query['COLLECTION_DATE'])) if MCL and slope < 0 else 'Not available'

        lower_date, upper_date = None, None
        if time_at_MCL and isinstance(time_at_MCL, date):
            lower_date, upper_date = self.adjust_date_based_on_confidence(time_at_MCL, slope, ts * std_err, 365.25)

        # Prepare for extrapolation
        extrapolate_date = np.datetime64(f'{extrapolate_until_year}-12-31')
        x_extrapolate = np.linspace(x.min(), date2num(extrapolate_date), 100)
        y_extrapolate = slope * x_extrapolate + intercept

        # Ensure all plots are of the same size
        plt.figure(figsize=(10, 6))
        plt.grid(False)

        data_points = plt.scatter(query['COLLECTION_DATE'], y, color='blue', label='Data Points')
        plt.xlim([num2date(x.min()), num2date(x_extrapolate.max())])
        plt.ylim([0.0, max(y.max(), y_extrapolate.max())])

        extrap_regression = plt.plot(num2date(x_extrapolate), y_extrapolate, 'r--', label=f'Extrapolated Regression')

        if log_MCL is not None:
            mcl_line = plt.axhline(y=log_MCL, color='green', linestyle='--', label=f'MCL (log10): {log_MCL:.2f}')

        if lower_date and upper_date:
            lower_intercept = intercept + slope * (date2num(lower_date) - date2num(time_at_MCL))
            upper_intercept = intercept + slope * (date2num(upper_date) - date2num(time_at_MCL))
            lower_ci = plt.plot(num2date(x_extrapolate), slope * x_extrapolate + lower_intercept, 'b--', label='Lower CI Line')
            upper_ci = plt.plot(num2date(x_extrapolate), slope * x_extrapolate + upper_intercept, 'orange', label='Upper CI Line')

        plt.xlabel('Collection Date')
        plt.ylabel('Log-Concentration (pCi/mL)')
        plt.title(f'{well_name} - {analyte_name}')

        handles, labels = plt.gca().get_legend_handles_labels()
        fig_legend = plt.legend(handles=handles, loc='upper center', bbox_to_anchor=(0.3, -0.2), fontsize=18, frameon=True, shadow=True)

        # Add p-value to the result string
        result_str = (
            f"Results\n"
            f"- Slope value: {slope:.2e}\n"
            f"- Time@MCL value: {time_at_MCL}\n"
            f"- Lower CI value: {lower_date}\n"
            f"- Upper CI value: {upper_date}\n"
            f"- p-value: {p_value:.4f}"
        )

        plt.figtext(0.55, -0.2, result_str, fontsize=18, ha="left", bbox=dict(facecolor='white', alpha=0.5))
        plt.tight_layout(rect=[0, 0.31, 1, 0.95])
        plt.gca().xaxis.set_major_locator(mdates.YearLocator(5))
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
        plt.gcf().autofmt_xdate()

        plt.xlim([num2date(x.min()), num2date(x_extrapolate.max())])

        figure_path = f'{save_dir}/{well_name}-{analyte_name}.png'
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        plt.savefig(figure_path)

        return figure_path

    def generate_analysis_dataframe(self, analyte_name_column, well_name_column, analyte_name, MCL):
        analyte_summary_df = self.generate_analyte_summary(analyte_name_column, analyte_name, well_name_column, MCL)
        analysis_data = []

        for index, row in analyte_summary_df.iterrows():
            if row['trend'] == 'Neutral':
                continue

            plot_path = self.plot_MCL(well_name_column, analyte_name_column, row['well_name'], analyte_name, year_interval=5, save_dir='plot_MCL', MCL=MCL)

            analysis_data.append({
                'analyte_name': analyte_name,
                'well_name': row['well_name'],
                'slope': row['slope'],
                'p_value': row['p_value'],  # Add p-value to analysis dataframe
                'trend': row['trend'],
                'time@MCL': row['time@MCL'],
                'lower_time@MCL': row['lower_time@MCL'],
                'upper_time@MCL': row['upper_time@MCL'],
                'total_decrease_duration': row['total_decrease_duration'],
                'path_to_figure': plot_path
            })

        return pd.DataFrame(analysis_data)

    def generate_decreasing_well_list(self, analyte_name_column, analyte_name, well_name_column, MCL):
        analyte_summary_df = self.generate_analyte_summary(analyte_name_column, analyte_name, well_name_column, MCL)
        decreasing_well_list = analyte_summary_df[analyte_summary_df['trend'] == 'Decrease']['well_name'].tolist()
        return decreasing_well_list

    def extract_location_data(self, well_name_column, decreasing_well_list):
        location_data = self.data[self.data[well_name_column].isin(decreasing_well_list)]
        return location_data

In [ ]:
data_analyzer = DataAnalyzer('https://raw.githubusercontent.com/ALTEMIS-DOE/pylenm/master/notebooks/data/FASB_Data_thru_3Q2015_Reduced_Demo.csv')
data_analyzer.print_column_names()  # This will print column names, helping you to identify the correct ones

# **Plot MCL for a specific well**

In [ ]:
data_analyzer.plot_MCL(
    well_name_column='STATION_ID',
    analyte_name_column='ANALYTE_NAME',
    well_name='FSP  2B',
    analyte_name='STRONTIUM-90',
    MCL=10,
    extrapolate_until_year=2025
)

# **Data Frame Table**

# ***Sr-90***

In [ ]:
analyte_summary_df_sr_90 = data_analyzer.generate_analyte_summary(
    analyte_name_column='ANALYTE_NAME',
    analyte_name='STRONTIUM-90',
    well_name_column='STATION_ID',
    MCL=10
)
print(analyte_summary_df_sr_90)

# ***I-129***

In [ ]:
analyte_summary_df_i_129 = data_analyzer.generate_analyte_summary(
    analyte_name_column='ANALYTE_NAME',
    analyte_name='IODINE-129',
    well_name_column='STATION_ID',
    MCL=10
)
print(analyte_summary_df_i_129)

# **Store Data Frame for a Specific Analyte**

# ***Sr-90***

In [ ]:
sr_90_data_frame = data_analyzer.generate_analysis_dataframe(
    'ANALYTE_NAME',
    'STATION_ID',
    'STRONTIUM-90',
    10)

# ***I-129***

In [ ]:
i_129_data_frame = data_analyzer.generate_analysis_dataframe(
    'ANALYTE_NAME',
    'STATION_ID',
    'IODINE-129',
    10)

# **Report**

In [ ]:
def report_of_data_frame(df, original_data, analyte_name):
    plt.style.use(['science', 'no-latex'])
    plt.grid(False)

    df['total_decrease_duration_years'] = df['total_decrease_duration'] / 365.25

    median_duration = df['total_decrease_duration_years'].median()

    counts, bins, patches = plt.hist(df['total_decrease_duration_years'], bins=20, edgecolor='b

    for i in range(len(patches)):
        if bins[i] < median_duration:
            patches[i].set_facecolor('blue')
        else:
            patches[i].set_facecolor('green')

    plt.axvline(median_duration, color='r', linestyle='--', label=f'Median: {median_duration:.2f} years')
    plt.title('Histogram of Total Decrease Duration (Years)', fontsize=14)  # Increase the font size
    plt.xlabel('Total Decrease Duration (Years)', fontsize=12)  # Increase the font size
    plt.ylabel('Frequency', fontsize=12)
    plt.legend(fontsize=12)
    plt.show()

    total_decrease_duration_years = df['total_decrease_duration_years'].sum()
    print(f"Overall Total Decrease Duration: {total_decrease_duration_years:.2f} years")
    trend_counts = df['trend'].value_counts()
    print(f"\nTrend counts:\n{trend_counts}")
    avg_decrease_duration_years = df[df['trend'] == 'Decrease']['total_decrease_duration_years'].mean()
    print(f"Average Decrease Duration for Decreasing Wells: {avg_decrease_duration_years:.2f} years")
    all_wells_count = original_data[original_data['ANALYTE_NAME'] == analyte_name]['STATION_ID'].nunique()
    print(f"\nTotal number of wells in the original dataset for analyte '{analyte_name}': {all_wells_count}")

    decreasing_wells = df[df['trend'] == 'Decrease']
    wells_less_than_10_years = decreasing_wells[decreasing_wells['total_decrease_duration_years'] < 10].shape[0]
    percentage_wells_less_than_10_years = (wells_less_than_10_years / decreasing_wells.shape[0]) * 100
    print(f"\nNumber of decreasing wells with time-to-MCL less than 10 years: {wells_less_than_10_years}")
    print(f"Percentage of decreasing wells with time-to-MCL less than 10 years: {percentage_wells_less_than_10_years:.2f}%")

In [ ]:
report_of_data_frame(sr_90_data_frame, data_analyzer.data, 'STRONTIUM-90')

In [ ]:
report_of_data_frame(i_129_data_frame, data_analyzer.data, 'IODINE-129')

In [ ]:
def compare_analytes_histogram(df1, df2, analyte1_name, analyte2_name):
    plt.style.use(['science', 'no-latex'])
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(8, 12))  # Changed layout to vertical

    df1['total_decrease_duration_years'] = df1['total_decrease_duration'] / 365.25
    df2['total_decrease_duration_years'] = df2['total_decrease_duration'] / 365.25

    df1_filtered = df1['total_decrease_duration_years'].dropna()
    df2_filtered = df2['total_decrease_duration_years'].dropna()

    median1 = df1_filtered.median()
    median2 = df2_filtered.median()

    counts1, bins1, patches1 = axes[0].hist(df1_filtered, bins=20, edgecolor='black')
    axes[0].set_title(f'(a)', fontsize=14)
    axes[0].set_xlabel('Total Decrease Duration (Years)', fontsize=12)
    axes[0].set_ylabel('Frequency', fontsize=12)
    for i in range(len(patches1)):
        if bins1[i] < median1:
            patches1[i].set_facecolor('blue')
        else:
            patches1[i].set_facecolor('green')
    axes[0].axvline(median1, color='r', linestyle='--', label=f'Median: {median1:.2f} years')
    axes[0].legend()

    counts2, bins2, patches2 = axes[1].hist(df2_filtered, bins=20, edgecolor='black')
    axes[1].set_title(f'(b)', fontsize=14)
    axes[1].set_xlabel('Total Decrease Duration (Years)', fontsize=12)
    axes[1].set_ylabel('Frequency', fontsize=12)
    for i in range(len(patches2)):
        if bins2[i] < median2:
            patches2[i].set_facecolor('blue')
        else:
            patches2[i].set_facecolor('green')
    axes[1].axvline(median2, color='r', linestyle='--', label=f'Median: {median2:.2f} years')
    axes[1].legend()

    def round_up_to_nearest(value, factor):
        return np.ceil(value / factor) * factor

    max_duration = max(bins1.max(), bins2.max())
    max_frequency = max(counts1.max(), counts2.max())
    max_duration_rounded = round_up_to_nearest(max_duration, 10)
    max_frequency_rounded = round_up_to_nearest(max_frequency, 5) + 5  # Add 5 as buffer
    axes[0].set_xlim(0, max_duration_rounded)
    axes[1].set_xlim(0, max_duration_rounded)
    axes[0].set_ylim(0, max_frequency_rounded)
    axes[1].set_ylim(0, max_frequency_rounded)

    plt.tight_layout()
    plt.show()

In [ ]:
compare_analytes_histogram(sr_90_data_frame, i_129_data_frame, 'STRONTIUM-90', 'IODINE-129')

In [ ]:
def plot_log_time_to_MCL(df, analyte_name, font_size=14, title_font_size=18):
    valid_mcl_data = df[df['total_decrease_duration'].notnull()]
    valid_mcl_data = valid_mcl_data[valid_mcl_data['total_decrease_duration'] > 0]
    valid_mcl_data['total_decrease_duration_years'] = valid_mcl_data['total_decrease_duration'] / 365.25
    valid_mcl_data['log_time_to_MCL'] = np.log10(valid_mcl_data['total_decrease_duration_years'])

    median_log_duration = valid_mcl_data['log_time_to_MCL'].median()

    plt.style.use(['science', 'no-latex'])  # Ensure SciencePlots is used
    plt.figure(figsize=(10, 6))
    counts, bins, patches = plt.hist(valid_mcl_data['log_time_to_MCL'], bins=20, edgecolor=None, alpha=0.7)

    for i in range(len(patches)):
        if bins[i] < median_log_duration:
            patches[i].set_facecolor('blue')
        else:
            patches[i].set_facecolor('green')

    plt.axvline(median_log_duration, color='r', linestyle='--', label=f'Median: {median_log_duration:.2f} (log-years)')
    plt.title(f'Log(Time-to-MCL) Distribution for {analyte_name}', fontsize=title_font_size)
    plt.xlabel('Log(Time-to-MCL in years)', fontsize=font_size)
    plt.ylabel('Frequency', fontsize=font_size)
    plt.xticks(fontsize=font_size)
    plt.yticks(fontsize=font_size)
    plt.grid(False)
    plt.legend(fontsize=12)
    plt.show()

    return valid_mcl_data[['well_name', 'total_decrease_duration_years', 'log_time_to_MCL']]

In [ ]:
log_time_to_mcl_sr_90 = plot_log_time_to_MCL(sr_90_data_frame, analyte_name='STRONTIUM-90', font_size=14, title_font_size=18)
print(log_time_to_mcl_sr_90)

In [ ]:
log_time_to_mcl_i_129 = plot_log_time_to_MCL(i_129_data_frame, analyte_name='IODINE-129', font_size=14, title_font_size=18)
print(log_time_to_mcl_i_129)

In [ ]:
def compare_log_time_to_MCL(df1, analyte1_name, df2, analyte2_name, font_size=14, title_font_size=18):
    valid_mcl_data1 = df1[df1['total_decrease_duration'].notnull() & (df1['total_decrease_duration'] > 0)]
    valid_mcl_data1['total_decrease_duration_years'] = valid_mcl_data1['total_decrease_duration'] / 365.25
    valid_mcl_data2 = df2[df2['total_decrease_duration'].notnull() & (df2['total_decrease_duration'] > 0)]
    valid_mcl_data2['total_decrease_duration_years'] = valid_mcl_data2['total_decrease_duration'] / 365.25

    valid_mcl_data1['log_time_to_MCL'] = np.log10(valid_mcl_data1['total_decrease_duration_years'])
    valid_mcl_data2['log_time_to_MCL'] = np.log10(valid_mcl_data2['total_decrease_duration_years'])

    median_log_duration1 = valid_mcl_data1['log_time_to_MCL'].median()
    median_log_duration2 = valid_mcl_data2['log_time_to_MCL'].median()
    mean_log_duration1 = valid_mcl_data1['log_time_to_MCL'].mean()
    mean_log_duration2 = valid_mcl_data2['log_time_to_MCL'].mean()

    plt.style.use(['science', 'no-latex'])  # Ensure SciencePlots is used
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 12))  # Vertical alignment

    counts1, bins1, patches1 = axes[0].hist(valid_mcl_data1['log_time_to_MCL'], bins=20, edgecolor=None, alpha=0.7)
    counts2, bins2, patches2 = axes[1].hist(valid_mcl_data2['log_time_to_MCL'], bins=20, edgecolor=None, alpha=0.7)

    for i in range(len(patches1)):
        if bins1[i] < median_log_duration1:
            patches1[i].set_facecolor('blue')
        else:
            patches1[i].set_facecolor('green')

    for i in range(len(patches2)):
        if bins2[i] < median_log_duration2:
            patches2[i].set_facecolor('blue')
        else:
            patches2[i].set_facecolor('green')

    axes[0].axvline(median_log_duration1, color='r', linestyle='--', label=f'Median: {median_log_duration1:.2f} (log-years)')
    axes[1].axvline(median_log_duration2, color='r', linestyle='--', label=f'Median: {median_log_duration2:.2f} (log-years)')
    axes[0].axvline(mean_log_duration1, color='skyblue', linestyle=':', label=f'Mean: {mean_log_duration1:.2f} (log-years)')
    axes[1].axvline(mean_log_duration2, color='skyblue', linestyle=':', label=f'Mean: {mean_log_duration2:.2f} (log-years)')
    axes[0].set_title(f'(a) {analyte1_name}', fontsize=title_font_size)
    axes[1].set_title(f'(b) {analyte2_name}', fontsize=title_font_size)
    axes[0].set_xlabel('Log(Time-to-MCL in years)', fontsize=font_size)
    axes[1].set_xlabel('Log(Time-to-MCL in years)', fontsize=font_size)
    axes[0].set_ylabel('Frequency', fontsize=font_size)
    axes[1].set_ylabel('Frequency', fontsize=font_size)
    axes[0].tick_params(axis='both', which='major', labelsize=font_size)
    axes[1].tick_params(axis='both', which='major', labelsize=font_size)

    def round_up_to_nearest(value, factor):
        return np.ceil(value / factor) * factor

    max_duration = max(bins1.max(), bins2.max())
    max_frequency = max(counts1.max(), counts2.max())
    max_duration_rounded = round_up_to_nearest(max_duration, 0.5)  # Adjust for log-scale bins
    max_frequency_rounded = round_up_to_nearest(max_frequency, 5) + 2.5  # Add 2.5 as a buffer

    axes[0].set_xlim(0, max_duration_rounded)
    axes[1].set_xlim(0, max_duration_rounded)
    axes[0].set_ylim(0, max_frequency_rounded)
    axes[1].set_ylim(0, max_frequency_rounded)
    axes[0].legend(fontsize=12)
    axes[1].legend(fontsize=12)

    plt.tight_layout()
    plt.show()

In [ ]:
log_time_to_mcl_comparative = compare_log_time_to_MCL(sr_90_data_frame, 'STRONTIUM-90', i_129_data_frame, 'IODINE-129', font_size=14, title_font_size=18)

In [ ]:
def combined_analyte_plots(df1, analyte1_name, df2, analyte2_name):
    plt.style.use(['science', 'no-latex'])
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(16, 12))  # 2x2 layout

    df1['total_decrease_duration_years'] = df1['total_decrease_duration'] / 365.25
    df2['total_decrease_duration_years'] = df2['total_decrease_duration'] / 365.25
    df1_filtered = df1['total_decrease_duration_years'].dropna()
    df2_filtered = df2['total_decrease_duration_years'].dropna()

    median1 = df1_filtered.median()
    median2 = df2_filtered.median()

    counts1, bins1, patches1 = axes[0, 0].hist(df1_filtered, bins=20, edgecolor='black')
    axes[0, 0].set_title(f'(a)', fontsize=14)
    axes[0, 0].set_xlabel('Total Decrease Duration (Years)', fontsize=12)
    axes[0, 0].set_ylabel('Frequency', fontsize=12)

    for i in range(len(patches1)):
        if bins1[i] < median1:
            patches1[i].set_facecolor('blue')
        else:
            patches1[i].set_facecolor('green')

    axes[0, 0].axvline(median1, color='r', linestyle='--', label=f'Median: {median1:.2f} years')
    axes[0, 0].legend()
    counts2, bins2, patches2 = axes[1, 0].hist(df2_filtered, bins=20, edgecolor='black')
    axes[1, 0].set_title(f'(b)', fontsize=14)
    axes[1, 0].set_xlabel('Total Decrease Duration (Years)', fontsize=12)
    axes[1, 0].set_ylabel('Frequency', fontsize=12)

    for i in range(len(patches2)):
        if bins2[i] < median2:
            patches2[i].set_facecolor('blue')
        else:
            patches2[i].set_facecolor('green')

    axes[1, 0].axvline(median2, color='r', linestyle='--', label=f'Median: {median2:.2f} years')
    axes[1, 0].legend()

    valid_mcl_data1 = df1[df1['total_decrease_duration'].notnull() & (df1['total_decrease_duration'] > 0)]
    valid_mcl_data1['total_decrease_duration_years'] = valid_mcl_data1['total_decrease_duration'] / 365.25
    valid_mcl_data1['log_time_to_MCL'] = np.log10(valid_mcl_data1['total_decrease_duration_years'])
    valid_mcl_data2 = df2[df2['total_decrease_duration'].notnull() & (df2['total_decrease_duration'] > 0)]
    valid_mcl_data2['total_decrease_duration_years'] = valid_mcl_data2['total_decrease_duration'] / 365.25
    valid_mcl_data2['log_time_to_MCL'] = np.log10(valid_mcl_data2['total_decrease_duration_years'])

    median_log_duration1 = valid_mcl_data1['log_time_to_MCL'].median()
    median_log_duration2 = valid_mcl_data2['log_time_to_MCL'].median()
    mean_log_duration1 = valid_mcl_data1['log_time_to_MCL'].mean()
    mean_log_duration2 = valid_mcl_data2['log_time_to_MCL'].mean()

    counts1_log, bins1_log, patches1_log = axes[0, 1].hist(valid_mcl_data1['log_time_to_MCL'], bins=20, edgecolor=None, alpha=0.7)
    axes[0, 1].set_title(f'(c)', fontsize=14)
    axes[0, 1].set_xlabel('Log10(Time-to-MCL in years)', fontsize=12)
    axes[0, 1].set_ylabel('Frequency', fontsize=12)

    for i in range(len(patches1_log)):
        if bins1_log[i] < median_log_duration1:
            patches1_log[i].set_facecolor('blue')
        else:
            patches1_log[i].set_facecolor('green')

    axes[0, 1].axvline(median_log_duration1, color='r', linestyle='--', label=f'Median: {median_log_duration1:.2f} (log-years)')
    axes[0, 1].axvline(mean_log_duration1, color='brown', linestyle=':', label=f'Mean: {mean_log_duration1:.2f} (log-years)')
    axes[0, 1].legend()
    counts2_log, bins2_log, patches2_log = axes[1, 1].hist(valid_mcl_data2['log_time_to_MCL'], bins=20, edgecolor=None, alpha=0.7)
    axes[1, 1].set_title(f'(d)', fontsize=14)
    axes[1, 1].set_xlabel('Log10(Time-to-MCL in years)', fontsize=12)
    axes[1, 1].set_ylabel('Frequency', fontsize=12)

    for i in range(len(patches2_log)):
        if bins2_log[i] < median_log_duration2:
            patches2_log[i].set_facecolor('blue')
        else:
            patches2_log[i].set_facecolor('green')

    axes[1, 1].axvline(median_log_duration2, color='r', linestyle='--', label=f'Median: {median_log_duration2:.2f} (log-years)')
    axes[1, 1].axvline(mean_log_duration2, color='brown', linestyle=':', label=f'Mean: {mean_log_duration2:.2f} (log-years)')
    axes[1, 1].legend()

    plt.tight_layout()
    plt.show()

In [ ]:
combined_analyte_plots(sr_90_data_frame, 'STRONTIUM-90', i_129_data_frame, 'IODINE-129')


# **Mapping**

In [ ]:
import folium
import branca.colormap as cm
from matplotlib import cm, colors as mcolors
from folium.plugins import MeasureControl
from geopy.distance import geodesic

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
location_analyzer = DataAnalyzer('/content/gdrive/MyDrive/MIT Project - VAL @HW_CEE/FASB Well Construction Info.xlsx')

In [ ]:
location_analyzer.print_column_names()

In [ ]:
def calculate_real_distances(min_latitude, max_latitude, min_longitude, max_longitude):
    # Calculate the distances in meters using geodesic distance
    width = geodesic((min_latitude, min_longitude), (min_latitude, max_longitude)).meters
    height = geodesic((min_latitude, min_longitude), (max_latitude, min_longitude)).meters
    return width, height

In [ ]:
def generate_map(df, duration_column, zoom_level_offset=10, satellite_mode=False, aquifer=None, min_latitude=None, max_latitude=None, min_longitude=None, max_longitude=None):
    if aquifer:
        df = df[df['aquifer'] == aquifer]

    if df.empty:
        print(f"No wells found for the aquifer: {aquifer}")
        return None

    if min_latitude is None:
        min_latitude = df['latitude'].min()
    if max_latitude is None:
        max_latitude = df['latitude'].max()
    if min_longitude is None:
        min_longitude = df['longitude'].min()
    if max_longitude is None:
        max_longitude = df['longitude'].max()

    center_latitude = (max_latitude + min_latitude) / 2
    center_longitude = (max_longitude + min_longitude) / 2

    max_delta = max(max_latitude - min_latitude, max_longitude - min_longitude)
    zoom_level = zoom_level_offset - int(np.log2(max_delta))  # Use the provided offset

    if satellite_mode:
        m = folium.Map(location=[center_latitude, center_longitude], zoom_start=zoom_level, tiles=None)
        folium.TileLayer(
            tiles='https://{s}.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
            attr='Google',
            name='Google Satellite',
            max_zoom=20,
            subdomains=['mt0', 'mt1', 'mt2', 'mt3']
        ).add_to(m)
    else:
        m = folium.Map(location=[center_latitude, center_longitude], zoom_start=zoom_level)

    conc_fg = folium.FeatureGroup(name="Concentrations")
    no_decline_fg = folium.FeatureGroup(name="No Decline")

    cmap_jet = cm.get_cmap('jet')
    num_colors = 6  # Optional number of colors
    color_list = [mcolors.rgb2hex(cmap_jet(i / num_colors)) for i in range(num_colors)]

    valid_df = df[df[duration_column].notnull() & (df[duration_column] > 0)]
    valid_df['duration_years'] = valid_df[duration_column] / 365.25
    valid_df['log_duration_years'] = np.log(valid_df['duration_years'])

    min_val_log = valid_df['log_duration_years'].min()
    max_val_log = valid_df['log_duration_years'].max()
    valid_df['normalized_log_duration'] = (valid_df['log_duration_years'] - min_val_log) / (max_val_log - min_val_log)

    cmap = mcolors.LinearSegmentedColormap.from_list('custom_colormap', color_list)

    for _, row in valid_df.iterrows():
        coordinates = [row['latitude'], row['longitude']]
        point_color = mcolors.rgb2hex(cmap(row['normalized_log_duration']))
        popup_content = f"""
        <div style='font-family: "Arial", sans-serif; font-size: 16px; max-width:800px;'>
        <ul>
        <li><b>Well Name:</b> {row['well_name']}</li>
        <li><b>Time to MCL:</b> {row[duration_column]} days ({row['duration_years']:.2f} Years)</li>
        <li><b>Aquifer:</b> {row['aquifer']}</li>
        </ul></div>
        """

        folium.Circle(
            radius=15,
            location=coordinates,
            popup=folium.Popup(popup_content, max_width=800),
            fill_color=point_color,
            color='#000000',
            fill_opacity=1,
            stroke=1,
            weight=1
        ).add_to(conc_fg)

    m.add_child(conc_fg)

    no_decline_df = df[df[duration_column].isnull() | (df[duration_column] <= 0)]

    for _, row in no_decline_df.iterrows():
        coordinates = [row['latitude'], row['longitude']]
        popup_content = f"""
        <div style='font-family: "Arial", sans-serif; font-size: 16px; max-width:800px;'>
        <ul>
        <li><b>Well Name:</b> {row['well_name']}</li>
        <li><b>Aquifer:</b> {row['aquifer']}</li>
        <li><b>No Decline Observed</b></li>
        </ul></div>
        """

        folium.Circle(
            radius=15,
            location=coordinates,
            popup=folium.Popup(popup_content, max_width=800),
            fill_color='black',
            color='#000000',
            fill_opacity=1,
            stroke=1,
            weight=1
        ).add_to(no_decline_fg)

    m.add_child(no_decline_fg)

    min_val = valid_df[duration_column].min()
    max_val = valid_df[duration_column].max()
    min_val_years = min_val / 365.25
    max_val_years = max_val / 365.25

    colormap_original = folium.LinearColormap(colors=color_list, vmin=min_val_years, vmax=max_val_years, caption=f"{duration_column} (years)")
    colormap_original.add_to(m)

    return m

## **Sr-90**

In [ ]:
decreasing_well_list = data_analyzer.generate_decreasing_well_list(
    analyte_name_column='ANALYTE_NAME',
    analyte_name='STRONTIUM-90',
    well_name_column='STATION_ID',
    MCL=10
)

location_data = location_analyzer.extract_location_data(
    well_name_column='station_id',
    decreasing_well_list=decreasing_well_list
)

merged_df_sr_90 = pd.merge(analyte_summary_df_sr_90, location_data, left_on='well_name', right_on='station_id', how='inner')

print(merged_df_sr_90)

In [ ]:
generate_map(merged_df_sr_90, 'total_decrease_duration')

In [ ]:
generate_map(merged_df_sr_90, 'total_decrease_duration', zoom_level_offset=11)

In [ ]:
generate_map(merged_df_sr_90, 'total_decrease_duration', satellite_mode=True)

In [ ]:
generate_map(merged_df_sr_90, 'total_decrease_duration', aquifer='UAZ_UTRAU', zoom_level_offset=9)

In [ ]:
generate_map(merged_df_sr_90, 'total_decrease_duration', aquifer='LAZ_UTRAU', zoom_level_offset=8)

## **I-129**

In [ ]:
decreasing_well_list = data_analyzer.generate_decreasing_well_list(
    analyte_name_column='ANALYTE_NAME',
    analyte_name='IODINE-129',
    well_name_column='STATION_ID',
    MCL=10
)

location_data = location_analyzer.extract_location_data(
    well_name_column='station_id',
    decreasing_well_list=decreasing_well_list
)

merged_df_i_129 = pd.merge(analyte_summary_df_i_129, location_data, left_on='well_name', right_on='station_id', how='inner')

print(merged_df_i_129)

In [ ]:
generate_map(merged_df_i_129, 'total_decrease_duration', zoom_level_offset=9)

In [ ]:
generate_map(merged_df_i_129, 'total_decrease_duration', satellite_mode=True)

In [ ]:
generate_map(merged_df_i_129, 'total_decrease_duration', aquifer='UAZ_UTRAU')

In [ ]:
generate_map(merged_df_i_129, 'total_decrease_duration', aquifer='LAZ_UTRAU')

# **Compare maps**

In [ ]:
def generate_aquifer_maps(df_sr_90, df_i_129, aquifers, duration_column, slope_column, p_value_column, zoom_level_offset=10, satellite_mode=False):
    maps = {}
    combined_df = pd.concat([df_sr_90, df_i_129])

    min_latitude = combined_df['latitude'].min()
    max_latitude = combined_df['latitude'].max()
    min_longitude = combined_df['longitude'].min()
    max_longitude = combined_df['longitude'].max()

    def create_map_for_analyte(df, analyte, aquifer):
        df_filtered = df[df['aquifer'] == aquifer]

        if df_filtered.empty:
            print(f"No wells found for analyte: {analyte} and aquifer: {aquifer}")
            return None

        center_latitude = (min_latitude + max_latitude) / 2
        center_longitude = (min_longitude + max_longitude) / 2

        max_delta = max(max_latitude - min_latitude, max_longitude - min_longitude)
        zoom_level = zoom_level_offset - int(np.log2(max_delta))  # Use the provided offset

        if satellite_mode:
            m = folium.Map(location=[center_latitude, center_longitude], zoom_start=zoom_level, tiles=None)
            folium.TileLayer(
                tiles='https://{s}.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
                attr='Google',
                name='Google Satellite',
                max_zoom=20,
                subdomains=['mt0', 'mt1', 'mt2', 'mt3']
            ).add_to(m)
        else:
            m = folium.Map(location=[center_latitude, center_longitude], zoom_start=zoom_level)

        cmap = cm.get_cmap('jet', 6)  # 6 discrete colors
        color_list = [mcolors.rgb2hex(cmap(i / 5)) for i in range(6)]  # Convert to hex colors

        slope_min = df_filtered[slope_column].min()
        slope_max = df_filtered[slope_column].max()

        df_filtered['normalized_slope'] = (df_filtered[slope_column] - slope_min) / (slope_max - slope_min)

        for _, row in df_filtered.iterrows():
            coordinates = [row['latitude'], row['longitude']]

            if row[p_value_column] <= 0.05:
                point_color = mcolors.rgb2hex(cmap(row['normalized_slope']))
            else:
                point_color = "#000000"  # Black for wells with no significant trend

            popup_content = f"""
            <div style='font-family: "Arial", sans-serif; font-size: 16px; max-width:800px;'>
            <ul>
            <li><b>Well Name:</b> {row['well_name']}</li>
            <li><b>Slope:</b> {row[slope_column]:.4f}</li>
            <li><b>P-value:</b> {row[p_value_column]:.4f}</li>
            <li><b>Time to MCL:</b> {row[duration_column]} days</li>
            <li><b>Aquifer:</b> {row['aquifer']}</li>
            </ul></div>
            """

            folium.Circle(
                radius=15,  # You can adjust the radius if needed
                location=coordinates,
                popup=folium.Popup(popup_content, max_width=800),
                fill_color=point_color,
                color='#000000',
                fill_opacity=1,
                stroke=True,
                weight=1
            ).add_to(m)

        colormap = folium.LinearColormap(colors=color_list, vmin=slope_min, vmax=slope_max, caption='Slope')
        colormap.add_to(m)

        return m

    for aquifer in aquifers:
        sr_90_map = create_map_for_analyte(df_sr_90, 'Sr-90', aquifer)
        if sr_90_map:
            maps[f"Sr-90_{aquifer}"] = sr_90_map

        i_129_map = create_map_for_analyte(df_i_129, 'I-129', aquifer)
        if i_129_map:
            maps[f"I-129_{aquifer}"] = i_129_map

    return maps

In [ ]:
aquifers = ['UAZ_UTRAU', 'LAZ_UTRAU']
maps = generate_aquifer_maps(merged_df_i_129, merged_df_i_129, aquifers, 'total_decrease_duration', 'slope', 'p_value')

In [ ]:
maps['Sr-90_UAZ_UTRAU']

In [ ]:
maps['Sr-90_LAZ_UTRAU']

In [ ]:
maps['I-129_UAZ_UTRAU']

In [ ]:
maps['I-129_LAZ_UTRAU']